## Import Libraries and Load dataset

In [ ]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import os

# Load the dataset from Colab
from google.colab import files
uploaded = files.upload()


Saving Health Dataset5.csv to Health Dataset5.csv


## Read dataset

In [ ]:
# Read the dataset
df = pd.read_csv('Health Dataset5.csv')

# Summary for general info
print("Summary for general info:")
df.info()

# Summary for descriptive statistics for numeric columns
print("\nSummary for descriptive statistics for numeric columns:")
print(df.describe())

# export and download file
df.to_csv("df.csv", index=False)

from google.colab import files
files.download("df.csv")


Summary for general info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17645 entries, 0 to 17644
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Country                        17645 non-null  object 
 1   Year                           17645 non-null  int64  
 2   Cost of a healthy diet         17504 non-null  float64
 3   Income                         17525 non-null  float64
 4   Inflation                      17590 non-null  float64
 5   Child mortality rate           17645 non-null  float64
 6   Unemployment Rate              17604 non-null  float64
 7   Life expectancy                17645 non-null  float64
 8   Incomplete tertiary education  17645 non-null  float64
 9   Gini coefficient               17525 non-null  float64
 10  Diabetes                       17615 non-null  float64
 11  BMI (female)                   17620 non-null  float64
 12  Cardiovascular disea

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#**Data Cleaning**

## Identify and Handle Duplicates

In [ ]:
# Identify duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Inspect duplicate rows
print(df[df.duplicated(keep=False)])

# Drop rows where all values are NaN (completely blank rows)
#df.dropna(how='all', inplace=True)


Number of duplicate rows: 0
Empty DataFrame
Columns: [Country, Year, Cost of a healthy diet, Income, Inflation, Child mortality rate, Unemployment Rate, Life expectancy, Incomplete tertiary education, Gini coefficient, Diabetes, BMI (female), Cardiovascular diseases, BMI (male), Sex ratio, GDP, Median age, CPI]
Index: []


## Handle Duplicates
Based on the above results, all the duplicates are blank rows (rows where all columns are NaN or empty), and I will drop those blank rows entirely

In [ ]:
# Drop rows where all values are NaN (all blank rows)
df.dropna(how='all', inplace=True)

# Drop duplicate rows (keep first occurrence)
df.drop_duplicates(inplace=True)

# Check remaining duplicates
print(f"Duplicates after dropping: {df.duplicated().sum()}")

Duplicates after dropping: 0


After removed the blank rows and verified there is no duplicates in this dataset.

##Identify Missing Data

In [ ]:
# Identify number of missing values per column

print("\nCount of missing values:")
print(df.isnull().sum())


Count of missing values:
Country                            0
Year                               0
Cost of a healthy diet           141
Income                           120
Inflation                         55
Child mortality rate               0
Unemployment Rate                 41
Life expectancy                    0
Incomplete tertiary education      0
Gini coefficient                 120
Diabetes                          30
BMI (female)                      25
Cardiovascular diseases           50
BMI (male)                        25
Sex ratio                          0
GDP                               50
Median age                         0
CPI                               54
dtype: int64


## Outliers Detection
The Interquartile Range (IQR) method is used for detecting outliers in this dataset. The reasons as follows:

IQR method is specifically apply for continuous numerical data as most variables in this dataset are continuous numerical, such as Inflation, GDP, CPI etc..

Additionally, IQR method is robust to skewness data, and some of the variables are high skewness, including Inflation, GDP, and CPI. This makes it more suitable than methods like z-score which assume normality.

Since the dataset has very low missing values (< 1.5%), the IQR method can be applied effectively without the need for complex imputation prior to outlier detection. Missing data will not significantly bias the quartile estimates.

The IQR method does not make assumptions for the data normal distribution as most of the variables are skewed, therefore, IQR is appropriate to apply for this dataset.

In [ ]:
# Check Outliers

# Iterate only through numeric columns
for col in df.select_dtypes(include='number').columns:
    # Ensure the column has numeric data before proceeding
    if pd.api.types.is_numeric_dtype(df[col]):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)]
        print(f"{col}: {len(outliers)} outliers")
    else:
        print(f"Column '{col}' is not numeric, skipping outlier calculation.")

Year: 0 outliers
Cost of a healthy diet: 434 outliers
Income: 1866 outliers
Inflation: 1801 outliers
Child mortality rate: 845 outliers
Unemployment Rate: 426 outliers
Life expectancy: 89 outliers
Incomplete tertiary education: 208 outliers
Gini coefficient: 370 outliers
Diabetes: 1205 outliers
BMI (female): 508 outliers
Cardiovascular diseases: 2708 outliers
BMI (male): 72 outliers
Sex ratio: 750 outliers
GDP: 2876 outliers
Median age: 52 outliers
CPI: 1494 outliers


## Impute missing values with Mean / Median / Mode Imputation for Training Set only

According to the Skewness Summary, approx zero used mean imputation; > 0.5 or < -0.5 used median imputation

Imputation apply to training set only, avoid data leakage

In [ ]:
# Imputation and Train-Test Split

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Columns to impute
mean_impute_cols = ['BMI (female)', 'BMI (male)']
median_impute_cols = [
    'CPI', 'Gini coefficient', 'Income', 'Inflation', 'Unemployment Rate', 'Diabetes',
    'Cardiovascular diseases', 'GDP', 'Cost of a healthy diet',
    'Incomplete tertiary education', 'Child mortality rate',
    'Life expectancy', 'Sex ratio', 'Median age'
]

# Containers for all-country data
train_dfs = []
test_dfs = []

# --- Per-country processing ---
for country in df['Country'].unique():
    df_country = df[df['Country'] == country].sort_values('Year').reset_index(drop=True)

    # Skip countries with very few rows
    if len(df_country) < 5:
        continue

    # Time-based train/test split (80% train)
    split_index = int(len(df_country) * 0.8)
    train_country = df_country.iloc[:split_index].copy()
    test_country = df_country.iloc[split_index:].copy()

    # --- Mean imputation ---
    for col in mean_impute_cols:
        if col in train_country.columns:
            mean_val = train_country[col].mean()
            if np.isnan(mean_val):
                mean_val = 0  # Fallback if all values are missing
            train_country[col].fillna(mean_val, inplace=True)
            test_country[col].fillna(mean_val, inplace=True)

    # --- Median imputation with fallback to (median - 1) or -1 ---
    for col in median_impute_cols:
        if col in train_country.columns:
            median_val = train_country[col].median()
            if np.isnan(median_val):
                fill_val = -1
            else:
                fill_val = median_val - 1
            train_country[col].fillna(fill_val, inplace=True)
            test_country[col].fillna(fill_val, inplace=True)

    # Add Country column explicitly before appending
    train_country['Country'] = country
    test_country['Country'] = country

    # Store per-country processed data
    train_dfs.append(train_country)
    test_dfs.append(test_country)

# Combine all countries into unified train/test sets
train_all = pd.concat(train_dfs, ignore_index=True)
test_all = pd.concat(test_dfs, ignore_index=True)

train_all = train_all.set_index(['Country', 'Year'])
test_all = test_all.set_index(['Country', 'Year'])

print(train_all.head())  # should now show Country and Year as index
print(train_all.index.names)  # ['Country', 'Year']

# Final check
print(" Missing values after imputation (Train):")
print(train_all.isnull().sum())

print("\n Missing values after imputation (Test):")
print(test_all.isnull().sum())

# export and download file
train_all.to_csv("train_all.csv", index=False)
train_all.to_csv("test_all.csv", index=False)

from google.colab import files
files.download("train_all.csv")
files.download("test_all.csv")




/tmp/ipython-input-11-1263988940.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_country[col].fillna(mean_val, inplace=True)
/tmp/ipython-input-11-1263988940.py:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)

                  Cost of a healthy diet  Income  Inflation  \
Country     Year                                              
Afghanistan 1950                    -1.0    -1.0    9.68342   
            1951                    -1.0    -1.0    9.68342   
            1952                    -1.0    -1.0    9.68342   
            1953                    -1.0    -1.0    9.68342   
            1954                    -1.0    -1.0    9.68342   

                  Child mortality rate  Unemployment Rate  Life expectancy  \
Country     Year                                                             
Afghanistan 1950             41.370100             6.9405          28.1563   
            1951             40.799400             6.9405          28.5836   
            1952             40.224000             6.9405          29.0138   
            1953             39.642300             6.9405          29.4521   
            1954             39.158897             6.9405          29.6975   

           

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The above result verify that all missing value have been imputed.

## Handling Outliers - Winsorization and Yeo-Johnson Transformation

1. Winsorization

Winsorization is a statistical technique that Capping extreme values at chosen percentiles to minimize the influence of outliers on data analysis, preserving the overall structure of the dataset, can be retains dataset integrity while reducing distortion.

It involves setting a threshold (e.g., the 5th and 95th percentiles) and replacing any values below the lower threshold with the value at that threshold, and any values above the upper threshold with the value at that threshold.

Several studies support Winsorization. Weichle (2023) investigated how different methods for handling outliers and influential observations impact the calculation of medical costs in a dataset and successfully apply Winsorization  to cap extreme cost values at the 5th and 95th percentiles to reduce the influence of extreme outliers Balia & Jones (2008): In colon cancer cost data, using Winsorization at 5% (5th–95th percentile) replaced 384 outliers, yielding a more consistent average cost without removing data
Carrascosa (2025): Provides a “complete guide” to handling outliers. Hoaglin & Iglewicz (1987) or Rousseeuw & Hubert (1991), both seminal works that recommend Winsorization in robust statistics.

Lu et al. (2024): Winsorization before RNA-seq analysis considerably reduced false positives, improving model performance, and was recommended at 95%

Pachter (2024) investigate the effective percentage of capping applying Winsorization, 93%, 95% and 987% are being tested. Concluded that use 95% for applying Winsorization is the best.

2. Yeo-Johnson Transformation

Yeo-Johnson transformation is used after handled outliers by Winsorization.
This dataset contains multiple continuous numeric variables (such as income, BMI, GDP, and health-related indicators) that show skewed distributions. Skewness causes coeficient bias, poor model fit, inefficient forecast and negatively impact many modeling techniques by violating assumptions such as ARIMA, linear regression and part of Prophet, that assume the features are normal distributed, therefore handle outliers is essential for generating reliable and stable predictions.

Yeo-Johnson transformation is used in this dataset. The Yeo-Johnson transformation is a statistical technique used to normalize data to make it more symmetrical and reduce skewness. The reason of using Yeo-Johnson because this method appropriate to work with continuous numeric varibles as this dataset primarily composed of continuous numerical features such as GDP, BMI, income, and other health indicators, exactly match the type of data Yeo-Johnson is designed to handle. Additionally, Yeo-Johnson transformation can handle positive, negative and zero values, which is suitable to handle this dataset that contains negative and zero values, such as inflation. Furthermore, Yeo-Johnson improve normality and reduce skewness, which appropriate to use as the assumption of  ARIMA, Prophet, linear regression require normal distributed residual, which enhances model validity and stablility.

Compatible with integer or float data
Yeo-Johnson can be applied to both integer and float types (e.g., "Median age"), eliminating the need for manual type conversion.

Not applicable to categorical variables
Your dataset includes one categorical feature (e.g., "Country"), which should be excluded from this transformation. Yeo-Johnson is only suitable for numeric features.

One of the categorical variable (e.g., Country) should be excluded
This transformation is not meant for categorical data — but that's fine. Just apply it only to your numeric columns.

Several studies have demonstrated the effectiveness of the Yeo-Johnson transformation in addressing these issues. For example, Zhang et al. (2018) applied the Yeo-Johnson transformation to normalize skewed biomarker and health outcome data prior to predictive modeling. Min et al. (2020) used the method to correct skewness in economic variables such as income and expenditure before conducting regression analysis. Similarly, Wang et al. (2019) employed Yeo-Johnson to transform environmental pollutant data, including values that were zero or negative, leading to improved model fit and interpretability. These studies provide strong evidence that Yeo-Johnson is a robust and versatile transformation suitable for datasets like yours.

Yeo-Johnson transformation benefits ARIMA and Prophet by improving normality and variance stability, helping assumptions and model fit.


In [ ]:
# Winsorization and Yeo_Johnson

import numpy as np
import pandas as pd
from sklearn.preprocessing import PowerTransformer

#  Additional Assign index: Country and Year
#df = df.set_index(['Country', 'Year'])

# List columns to transform (excluding only identifiers)
exclude_cols = ['Country', 'Year', 'Life expectancy', 'Diabetes', 'Cardiovascular diseases']
target_cols = ['Life expectancy', 'Diabetes', 'Cardiovascular diseases']
numeric_cols = [
    col for col in train_all.columns
    if col not in exclude_cols
]

# --- Step 1: Winsorization at 5‑95% ---
def winsorize_df(df, cols, lower_q=0.05, upper_q=0.95):
    df_w = train_all.copy()
    limits = {}
    for col in cols:
        lower = train_all[col].quantile(lower_q)
        upper = train_all[col].quantile(upper_q)
        limits[col] = (lower, upper)
        df_w[col] = np.clip(df[col], lower, upper)
    return df_w, limits

# Apply Winsorization to train set
train_df_w, limits = winsorize_df(train_all, numeric_cols, 0.05, 0.95)

# Apply same limits to test set
test_df_w = test_all.copy()
for col, (low, high) in limits.items():
    test_df_w[col] = np.clip(test_all[col], low, high)

# --- Step 2: Yeo‑Johnson Transformation ---
pt = PowerTransformer(method='yeo-johnson', standardize=False)

# Fit on winsorized train data
train_df_transformed = train_df_w.copy()
train_df_transformed[numeric_cols] = pt.fit_transform(train_df_w[numeric_cols])

# Apply to test data
test_df_transformed = test_df_w.copy()
test_df_transformed[numeric_cols] = pt.transform(test_df_w[numeric_cols])

# --- Preview results ---
print("=== Train Transformed Sample ===")
print(train_df_transformed[target_cols + [c for c in numeric_cols if c not in target_cols]].head())
print("\n=== Test Transformed Sample ===")
print(test_df_transformed[target_cols + [c for c in numeric_cols if c not in target_cols]].head())

# combine train and test
# --- Step 3: Combine Transformed Train and Test Data ---
df_transformed = pd.concat([train_df_transformed, test_df_transformed], axis=0)

print("\n✅ Combined Transformed DataFrame:")
print(df_transformed.head())

# export and download file
df_transformed.to_csv("df_transformed.csv", index=False)
train_df_transformed.to_csv("train_df_transformed.csv", index=False)
test_df_transformed.to_csv("test_df_transformed.csv", index=False)

from google.colab import files
files.download("df_transformed.csv")
files.download("train_df_transformed.csv")
files.download("test_df_transformed.csv")


=== Train Transformed Sample ===
                  Life expectancy  Diabetes  Cardiovascular diseases  \
Country     Year                                                       
Afghanistan 1950          28.1563       6.2                  3.97278   
            1951          28.5836       6.2                  3.97278   
            1952          29.0138       6.2                  3.97278   
            1953          29.4521       6.2                  3.97278   
            1954          29.6975       6.2                  3.97278   

                  Cost of a healthy diet    Income  Inflation  \
Country     Year                                                
Afghanistan 1950                0.871262  1.069802   2.119237   
            1951                0.871262  1.069802   2.119237   
            1952                0.871262  1.069802   2.119237   
            1953                0.871262  1.069802   2.119237   
            1954                0.871262  1.069802   2.119237   

      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>